In [2]:

from __future__ import division
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import control
import control.matlab as mlab
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as sla
import scipy.integrate as integrate
from sympy import Matrix
import numpy.linalg as nla
import matplotlib as mpl
from _547utils import *
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=2)
lw=4
fs=20

In [9]:
def isPSD(P, verbose=True):
    if np.allclose(P,P.T, rtol=1e-3) and np.all(np.linalg.eig(P)[0]>0):
        print('is PD?  :   True')
        if verbose:
            print('eigs(P) :  ', np.linalg.eig(P)[0])
    else: print('is PD?  :   False')

## Controllability 
### Continuous Time 
To check controllability there are three tests:
* matrix rank: $$\mathrm{rank}(\mathcal{C})=\mathrm{rank}\left(\begin{bmatrix} B & AB & \cdots  &A^{n-1}B\end{bmatrix}\right)=n?$$
* PBH: at $s\in \text{spec}(A)$, $$\mathrm{rank}\left(\begin{bmatrix} sI-A & B\end{bmatrix}\right)=n?$$
* Lyapunov test: if $A$ is stable, $(A,B)$ is controllable iff $\exists!\ W=W^\top >0$ s.t. $$AW+WA^\top=-BB^\top$$

In [66]:
n=4 # system dimensions
m= 2
np.random.seed(2)
A=np.random.rand(n,n)-2*np.eye(n) # generate random stable matrix
print("A:\n", A)
B=np.random.rand(n,m)
print("B:\n", B)
print("eig(A):\n", np.linalg.eig(A)[0])
print("")
Id=np.eye(n)

A:
 [[-1.56  0.03  0.55  0.44]
 [ 0.42 -1.67  0.2   0.62]
 [ 0.3   0.27 -1.38  0.53]
 [ 0.13  0.51  0.18 -1.21]]
B:
 [[0.85 0.49]
 [0.85 0.08]
 [0.51 0.07]
 [0.43 0.1 ]]
eig(A):
 [-0.4 +0.j   -1.92+0.14j -1.92-0.14j -1.58+0.j  ]



### Let's do the matrix rank test

In [74]:
def isCTRB(C, n):
    if np.linalg.matrix_rank(C)==n:
        print('is CC?   :   True')
        print("rank(C): ", np.linalg.matrix_rank(C))
    else: print('is CC?   :   False')


In [75]:
n=A.shape[0]
C_=B
for i in range(n-1):
    A_=np.copy(A)
    for j in range(i): A_=A_@A
    C_=np.hstack((C_, A_@B))
    
# use the control toolbox
C=control.ctrb(A,B)
print(np.allclose(C,C_, rtol=1e-3)) # check that they are teh same
isCTRB(C, n)
isCTRB(C_, n)


True
is CC?   :   True
rank(C):  4
is CC?   :   True
rank(C):  4


#### Let's do the PBH test

In [76]:
## this is a simple function to implement the PBH test

def PBH(A):
    '''PBH test
    input: dynamics A, nxn array
    output: true if rank(sI-A|B)=n for all s in spec(A)
            false otherwise
    '''
    print(np.all(np.array([np.linalg.matrix_rank(np.hstack((s*np.eye(A.shape[0])-A, B)))-A.shape[0] for s in np.linalg.eig(A)[0]])<=1e-4))

    

In [77]:
PBH(A)

True


#### Let's do the Lyapunov test

In [78]:


Q=-B@B.T
W=sla.solve_lyapunov(A.T,Q)
print("W:\n",W)
print("")
isPSD(W)

W:
 [[0.6  0.51 0.48 0.65]
 [0.51 0.49 0.45 0.61]
 [0.48 0.45 0.44 0.61]
 [0.65 0.61 0.61 0.89]]

is PD?  :   True
eigs(P) :   [2.31e+00 8.30e-02 1.98e-02 8.32e-04]


## Now that you see how to do this for Controllability, can you create similar functions for observability?

### Continuous Time 
To check observability there are three tests:
* matrix rank: $$\mathrm{rank}(\mathcal{O})=\mathrm{rank}\left(\begin{bmatrix} C \\ CA\\ \vdots \\CA^{n-1}\end{bmatrix}\right)=n?$$
* PBH: at $s\in \text{spec}(A)$, $$\mathrm{rank}\left(\begin{bmatrix} sI-A\\ C\end{bmatrix}\right)=n?$$
* Lyapunov test: if $A$ is stable, $(C,A)$ is controllable iff $\exists!\ P=P^\top >0$ s.t. $$A^\top P+PA=-C^\top C$$

## Next Up: Can we design an input to stabilize?
After we talk about controllability, we will introduce the notion of stabilizability and how to leverage Lyapunov theory to design feedback stabilizing controller.